# Hackathon AI4S - Carbon Estimation 

This notebook serves as a basis for the first part of the Hackathon, where you are tasked to create a tool to estimate a carbon footprint - use it as you see fit!

## Install Libraries

In [1]:
# Uncomment if needed to install libraries - feel free to use any library you want

#pip install pandas
#pip install plotly
#pip install scikit-learn
#...

## Import Libraries

In [1]:
import pandas as pd 
import plotly.express as px

Documentation Plotly
https://plotly.com/python/

## 1) Import Data

### 1.1) Import train data

The training data that will enable us to calculate the emissions are open source data extracted from ADEME for one part and scraped from the Climate Act website for the other.

ADEME : https://bilans-ges.ademe.fr/

Climate Act link : https://passerelle.sustainsoft.eu/climate-act

In [2]:
df_train = pd.read_excel('Data_train.xlsx')
df_train = df_train.drop(columns=["Unnamed: 0"])

In [3]:
df_train.head()

,entity_name,date,date_publication,entity_employees,entity_sales,source,siren,description_en,sector_naf_level_0_details,sector_naf_level_1_details,...,poste_14,poste_15,poste_16,poste_17,poste_18,poste_19,poste_20,poste_21,poste_22,poste_23
0,Labeyrie SAS,2016,2016-12-26,1450,290.4,ADEME,347902587,"Processing, conservation and marketing of aqua...",C. Industrie Manufacturière,10 - Industries alimentaires,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,NOVANDIE SA,2018,2019-12-20,1231,NaN,ADEME,314603051,Manufacture of liquid milk and fresh products,C. Industrie Manufacturière,10 - Industries alimentaires,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,HSBC FRANCE,2019,2020-12-16,7645,NaN,ADEME,775670284,BANKING ACTIVITY,K. Activité financières et d'assurance,"64 - Activités des services financiers, hors a...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Jamespot,2015,2017-05-04,15,3.1651,ADEME,483321378,French software editor specialized in collabo...,J. Information et communication,58 - Édition,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040418,0.0
4,Vilogia,2018,2020-04-06,933,NaN,ADEME,475680815,Vilogia is an entrepreneurial social real esta...,L. Activités immobilières,68 - Activités immobilières,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004143,0.0


* entity_sales are in Millions €

In [4]:
print('Columns from the dataset are the following :"', [column for column in df_train.columns])

Columns from the dataset are the following :" ['entity_name', 'date', 'date_publication', 'entity_employees', 'entity_sales', 'source', 'siren', 'description_en', 'sector_naf_level_0_details', 'sector_naf_level_1_details', 'sector_naf_level_2', 'region', 'city', 'emissions_total', 'scope_1', 'scope_2', 'scope_3', 'poste_1', 'poste_2', 'poste_3', 'poste_4', 'poste_5', 'poste_6', 'poste_7', 'poste_8', 'poste_9', 'poste_10', 'poste_11', 'poste_12', 'poste_13', 'poste_14', 'poste_15', 'poste_16', 'poste_17', 'poste_18', 'poste_19', 'poste_20', 'poste_21', 'poste_22', 'poste_23']


What is a NAF and SIREN code ?

Documentation NAF : https://www.insee.fr/fr/information/2406147

Documentation SIREN : https://fr.wikipedia.org/wiki/Syst%C3%A8me_d%27identification_du_r%C3%A9pertoire_des_entreprises

### 1.2) Import test data

The test dataset is composed of companies for which we wish to know an estimate of the tons of CO2 emitted by them. We call it test dataset because it's the one on which we evaluate the prediction model.

WARNING: the features available are not necessarily available in the training database and vice versa.

In [5]:
df_test  = pd.read_excel('Data_test.xlsx')
df_test  = df_test.drop(columns=["Unnamed: 0"])

In [6]:
df_test.head()

,entity_name,date,date_publication,entity_employees,entity_sales,source,siren,description_en,sector_naf_level_0_details,sector_naf_level_1_details,sector_naf_level_2,region,city
0,Rector Lesage,2015,2017-01-06,708,185.1414,ADEME,307322214.0,Manufacturing and marketing of concrete elemen...,C. Industrie Manufacturière,23 - Fabrication d'autres produits minéraux no...,2361Z,NaN,MULHOUSE
1,Brandt France,2018,2019-12-20,954,180.0538,ADEME,801250531.0,Manufacture of household appliances,G. Commerce; réparation d'automobiles et de mo...,"46 - Commerce de gros, à l’exception des autom...",4643Z,NaN,Rueil Malmaison
2,Caisse Régionale d'Assurance Maladie d'Ile de ...,2018,2019-12-18,1717,NaN,ADEME,775694730.0,The Cramif: pays benefits (disability pension...,O. Administration publique,84 - Administration publique et défense ; sécu...,8430A,NaN,Paris
3,KIMBERLY CLARK SAS,2017,2019-07-08,486,187.6831,ADEME,352600456.0,consumer products for personal hygiene and ca...,C. Industrie Manufacturière,17 - Industrie du papier et du carton,1722Z,NaN,NANTERRE
4,SOPREMA ENTREPRISES,2019,2020-06-18,1996,NaN,ADEME,485197552.0,SOPREMA Entreprises is a network of several do...,F. Construction,43 - Travaux de construction spécialisés,4399A,NaN,Strasbourg


* The social capital is in €.

In [7]:
print('Columns from the dataset are the following :"', [column for column in df_test.columns])

Columns from the dataset are the following :" ['entity_name', 'date', 'date_publication', 'entity_employees', 'entity_sales', 'source', 'siren', 'description_en', 'sector_naf_level_0_details', 'sector_naf_level_1_details', 'sector_naf_level_2', 'region', 'city']


## 2) Data Exploration

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   entity_name                 1261 non-null   object        
 1   date                        1261 non-null   int64         
 2   date_publication            1251 non-null   datetime64[ns]
 3   entity_employees            1261 non-null   int64         
 4   entity_sales                811 non-null    object        
 5   source                      1261 non-null   object        
 6   siren                       1261 non-null   object        
 7   description_en              1261 non-null   object        
 8   sector_naf_level_0_details  1261 non-null   object        
 9   sector_naf_level_1_details  1261 non-null   object        
 10  sector_naf_level_2          1261 non-null   object        
 11  region                      1 non-null      object      

With the .info() method we see that we miss a lot of information in some columns.

In [9]:
px.bar(df_train, x = df_train['sector_naf_level_0_details'].value_counts().index, y = df_train['sector_naf_level_0_details'].value_counts().values, title= "Number of instances per NAF macrosector" )

In [10]:
px.box(df_train, y =df_train['emissions_total'], x = df_train['sector_naf_level_0_details'],  height = 800, title= " Emissions distributions per NAF macrosector" )

#you can zoom on the graph if needed

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   entity_name                 315 non-null    object        
 1   date                        315 non-null    int64         
 2   date_publication            311 non-null    datetime64[ns]
 3   entity_employees            315 non-null    int64         
 4   entity_sales                189 non-null    object        
 5   source                      315 non-null    object        
 6   siren                       314 non-null    float64       
 7   description_en              315 non-null    object        
 8   sector_naf_level_0_details  315 non-null    object        
 9   sector_naf_level_1_details  315 non-null    object        
 10  sector_naf_level_2          315 non-null    object        
 11  region                      0 non-null      float64       

In [12]:
px.bar(df_test, x = df_test['sector_naf_level_0_details'].value_counts().index, y = df_test['sector_naf_level_0_details'].value_counts().values, title="emissions distribution per NAF macrosector")

To explore more the data you can plot more graphs (that you might add on your presentation support)

After the data exploration phase you should start predicting the carbon emissions for each company of the test dataset.

For this you might only choose a part of the dataset based on the discriminative features (variables, columns).

It can be for instance : only B2C companies, only greentechs, only one sector, one city, one type of revenue model ... it's up to you and your teammates.

## 3) Modelisation

### 3.1) Baseline (basic model)

In order to get a first naive estimation of a carbon footprint, you can take the average emissions per sector and per employees.

In [13]:
df_train['CO2 intensity (/emp)'] = df_train['emissions_total']/df_train['entity_employees'] #here we choose to predict the carbon intensity per employee

In [14]:
df_sectors_carbon_intensity = df_train.groupby('sector_naf_level_0_details').mean()[['CO2 intensity (/emp)']] #we create a new dataset to group all the instance per macrosector

df_sectors_carbon_intensity

,CO2 intensity (/emp)
sector_naf_level_0_details,
"A. Agriculture, sylviculture et pêche",221.538115
C. Industrie Manufacturière,437.721236
"D. Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné",55.964285
"E. Production et distribution d'eau; assainissement, gestion des déchets et dépollution",181.432913
F. Construction,291.492989
G. Commerce; réparation d'automobiles et de motocycles,591.523073
H. Transport et entreposage,1330.395541
I. Hébergement et restauration,13.208875
J. Information et communication,175.673683


In [27]:
df_results  = df_test.merge(df_sectors_carbon_intensity, how = 'left', left_on = 'sector_naf_level_1_details', right_index = True) #we merge the two dataframes in order to get the predictions for each company

#we create a new column that takes the average carbon intensity per employee of the firm sector and multiply it with the accurate number of employees
df_results['emissions_total'] = df_results['entity_employees']*df_results['CO2 intensity (/emp)'] 

#show the results
df_results.head()

,entity_name,date,date_publication,entity_employees,entity_sales,source,siren,description_en,sector_naf_level_0_details,sector_naf_level_1_details,sector_naf_level_2,region,city,CO2 intensity (/emp),emissions_total
0,Rector Lesage,2015,2017-01-06,708,185.1414,ADEME,307322214.0,Manufacturing and marketing of concrete elemen...,C. Industrie Manufacturière,23 - Fabrication d'autres produits minéraux no...,2361Z,NaN,MULHOUSE,NaN,NaN
1,Brandt France,2018,2019-12-20,954,180.0538,ADEME,801250531.0,Manufacture of household appliances,G. Commerce; réparation d'automobiles et de mo...,"46 - Commerce de gros, à l’exception des autom...",4643Z,NaN,Rueil Malmaison,NaN,NaN
2,Caisse Régionale d'Assurance Maladie d'Ile de ...,2018,2019-12-18,1717,NaN,ADEME,775694730.0,The Cramif: pays benefits (disability pension...,O. Administration publique,84 - Administration publique et défense ; sécu...,8430A,NaN,Paris,NaN,NaN
3,KIMBERLY CLARK SAS,2017,2019-07-08,486,187.6831,ADEME,352600456.0,consumer products for personal hygiene and ca...,C. Industrie Manufacturière,17 - Industrie du papier et du carton,1722Z,NaN,NANTERRE,NaN,NaN
4,SOPREMA ENTREPRISES,2019,2020-06-18,1996,NaN,ADEME,485197552.0,SOPREMA Entreprises is a network of several do...,F. Construction,43 - Travaux de construction spécialisés,4399A,NaN,Strasbourg,NaN,NaN


In [50]:
#df_results.to_excel('results baseline.xlsx')

What can you deduce from those results ?

For a more complex model, think about what you want to predict? What metric should be optimised? Which input variables? What is the chosen scope?

Reminder : you want to calculate the TOTAL emissions for a company. We gave you the amount for each enterprises included in the df_train. Maybe this feature can be decomposed if you see some correlation with other features. 

### 3.2) Machine Learning models

Machine Learning Regression Algorithm : 
https://www.analyticsvidhya.com/blog/2021/05/5-regression-algorithms-you-should-know-introductory-guide/

Documentation Linear Regression :
- how to use the function with sklearn documentation : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Documentation Decision Tree :
- how to use the function with sklearn documentation : https://scikit-learn.org/stable/modules/tree.html#regression

Documentation Random Forest :
- how to use the function with sklearn documentation : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Documentation Support Vector Regression :
- how to use the function with sklearn documentation : https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [28]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [30]:
# In order to use a textual feature you must encode it before using it in a model
def onehotencoder(df_train_,df_test_,column):
    encoder = OneHotEncoder(handle_unknown='ignore')
    #perform one-hot encoding on 'team' column 
    encoder_df_train = pd.DataFrame(encoder.fit_transform(df_train_[[column]]).toarray(),columns=encoder.categories_)
    encoder_df_test = pd.DataFrame(encoder.fit_transform(df_test_[[column]]).toarray(),columns=encoder.categories_)
    #merge one-hot encoded columns back with original DataFrame
    df_train_ = df_train_.join(encoder_df_train)
    df_test_ = df_test_.join(encoder_df_test)
    return(df_train_.drop(columns = [column]),df_test_.drop(columns = [column]))

Documentation Scaler :
- how to use the fonction with sklearn documentation : https://www.geeksforgeeks.org/data-pre-processing-wit-sklearn-using-standard-and-minmax-scaler/#:~:text=Data%20Scaling%20is%20a%20data%20preprocessing%20step%20for,results.%20Various%20scalers%20are%20defined%20for%20this%20purpose.

In [ ]:
#data preprocessing AFTER encoding
scaler  = '...' #pick a scaler
scaler.fit_transform(df_test)

You can fine tune you're model if you have the time. Reminder : the performance and the technical part only coutns for 20% of the final grade.